# PMI Korea SNS 콘텐츠 OCR 및 음성인식 처리 노트북

## 🚀 실행 전 체크리스트:
1. 런타임 > 런타임 유형 변경 > **GPU (T4 권장)**
2. `extracted_image_urls.csv` 준비
3. `extracted_video_urls.csv` 준비
4. 각 셀을 **순서대로** 실행 (Shift+Enter)

## ⏱️ 예상 소요 시간:
- 이미지 OCR: ~40-50분 (1,875개)
- 비디오 프레임 OCR: ~8-12분 (61개)
- 유튜브 자막: ~3-5분 (104개)
- Whisper 음성인식: ~25-35분

**총 예상: 약 1.5시간**

## [셀 1] GPU 확인 및 기본 설정

In [ ]:
import os
import sys
from pathlib import Path

# GPU 확인
!nvidia-smi

# 작업 디렉토리 생성
!mkdir -p /content/pmi_data
!mkdir -p /content/pmi_data/results
!mkdir -p /content/pmi_data/temp_videos

print("\n✅ GPU 확인 완료!")
print("📁 작업 디렉토리 생성 완료!")

## [셀 2] 라이브러리 설치

In [ ]:
# OCR 라이브러리 설치
!pip install -q easyocr

# 비디오 처리 라이브러리
!pip install -q opencv-python-headless
!pip install -q yt-dlp

# 음성인식 (Whisper)
!pip install -q openai-whisper

# 유튜브 자막 API
!pip install -q youtube-transcript-api

# 기타 유틸리티
!pip install -q tqdm pandas pillow requests

print("✅ 모든 라이브러리 설치 완료!")

## [셀 3] 파일 업로드 (CSV 파일들)

In [ ]:
from google.colab import files
import pandas as pd

print("📤 extracted_image_urls.csv 파일을 업로드하세요...")
uploaded = files.upload()

# 파일 이동
for filename in uploaded.keys():
    !mv {filename} /content/pmi_data/

print("\n📤 extracted_video_urls.csv 파일을 업로드하세요...")
uploaded = files.upload()

for filename in uploaded.keys():
    !mv {filename} /content/pmi_data/

# 파일 확인
image_df = pd.read_csv('/content/pmi_data/extracted_image_urls.csv')
video_df = pd.read_csv('/content/pmi_data/extracted_video_urls.csv')

print(f"\n✅ 업로드 완료!")
print(f"   - 이미지: {len(image_df)}개")
print(f"   - 비디오: {len(video_df)}개")

## [셀 4] OCR 및 처리 함수 정의

In [ ]:
import easyocr
import cv2
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import warnings
warnings.filterwarnings('ignore')

# EasyOCR 리더 초기화 (GPU 사용, 한국어+영어)
print("🔧 EasyOCR 초기화 중... (최초 실행 시 모델 다운로드로 1-2분 소요)")
reader = easyocr.Reader(['ko', 'en'], gpu=True)
print("✅ EasyOCR 준비 완료!")

def download_image(url, timeout=10):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
            'Referer': 'https://blog.naver.com/'
        }
        response = requests.get(url, headers=headers, timeout=timeout)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content))
        return np.array(image)
    except Exception as e:
        return None

def perform_ocr(image_array):
    try:
        results = reader.readtext(image_array)
        texts = [result[1] for result in results]
        combined_text = ' '.join(texts)
        if results:
            avg_confidence = sum([result[2] for result in results]) / len(results)
        else:
            avg_confidence = 0.0
        return combined_text, avg_confidence
    except Exception as e:
        return "", 0.0

def extract_first_frame(video_path):
    try:
        cap = cv2.VideoCapture(video_path)
        for _ in range(5):
            cap.read()
        ret, frame = cap.read()
        cap.release()
        if ret:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            return frame_rgb
        else:
            return None
    except Exception as e:
        return None

print("✅ 함수 정의 완료!")

## [셀 5] 이미지 OCR 일괄 처리
**예상 시간: 40-50분** ☕

In [ ]:
from tqdm.notebook import tqdm
import time

print("="*70)
print("📸 이미지 OCR 처리 시작")
print("="*70)

image_results = []

for idx, row in tqdm(image_df.iterrows(), total=len(image_df), desc="이미지 OCR"):
    post_id = row['post_id']
    url = row['url']
    image = download_image(url)
    
    if image is not None:
        ocr_text, confidence = perform_ocr(image)
        image_results.append({
            'post_id': post_id,
            'url': url,
            'ocr_text': ocr_text,
            'confidence': confidence,
            'status': 'success'
        })
    else:
        image_results.append({
            'post_id': post_id,
            'url': url,
            'ocr_text': '',
            'confidence': 0.0,
            'status': 'download_failed'
        })
    time.sleep(0.1)

image_ocr_df = pd.DataFrame(image_results)
success_count = (image_ocr_df['status'] == 'success').sum()
with_text = (image_ocr_df['ocr_text'].str.len() > 0).sum()

print(f"\n✅ 이미지 OCR 완료!")
print(f"   - 성공: {success_count}/{len(image_df)}")
print(f"   - 텍스트 추출: {with_text}개")
print(f"   - 평균 신뢰도: {image_ocr_df['confidence'].mean():.2%}")

image_ocr_df.to_csv('/content/pmi_data/results/image_ocr_results.csv', index=False, encoding='utf-8-sig')
print(f"💾 중간 결과 저장: image_ocr_results.csv")

## [셀 6] 비디오 첫 프레임 OCR
**예상 시간: 8-12분**

In [ ]:
print("="*70)
print("🎬 비디오 첫 프레임 OCR 처리 시작")
print("="*70)

video_frame_results = []
naver_videos = video_df[video_df['type'] == 'naver_blog'].copy()

print(f"처리 대상: 네이버 블로그 비디오 {len(naver_videos)}개\n")

for idx, row in tqdm(naver_videos.iterrows(), total=len(naver_videos), desc="비디오 프레임 OCR"):
    post_id = row['post_id']
    url = row['url']
    
    try:
        temp_video = f"/content/pmi_data/temp_videos/video_{idx}.mp4"
        headers = {'User-Agent': 'Mozilla/5.0', 'Referer': 'https://blog.naver.com/'}
        response = requests.get(url, headers=headers, timeout=30, stream=True)
        
        if response.status_code == 200:
            with open(temp_video, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            
            frame = extract_first_frame(temp_video)
            if frame is not None:
                ocr_text, confidence = perform_ocr(frame)
                video_frame_results.append({
                    'post_id': post_id, 'url': url, 'frame_ocr_text': ocr_text,
                    'confidence': confidence, 'status': 'success'
                })
            else:
                video_frame_results.append({
                    'post_id': post_id, 'url': url, 'frame_ocr_text': '',
                    'confidence': 0.0, 'status': 'frame_extraction_failed'
                })
            if os.path.exists(temp_video):
                os.remove(temp_video)
        else:
            video_frame_results.append({'post_id': post_id, 'url': url, 'frame_ocr_text': '', 'confidence': 0.0, 'status': 'download_failed'})
    except Exception as e:
        video_frame_results.append({'post_id': post_id, 'url': url, 'frame_ocr_text': '', 'confidence': 0.0, 'status': f'error: {str(e)[:50]}'})
    time.sleep(0.2)

if video_frame_results:
    video_frame_df = pd.DataFrame(video_frame_results)
    video_frame_df.to_csv('/content/pmi_data/results/video_frame_ocr_results.csv', index=False, encoding='utf-8-sig')
    success = (video_frame_df['status'] == 'success').sum()
    print(f"\n✅ 비디오 프레임 OCR 완료! 성공: {success}/{len(naver_videos)}")
    print(f"💾 저장: video_frame_ocr_results.csv")

## [셀 7] 유튜브 자막 추출
**예상 시간: 3-5분**

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled, NoTranscriptFound

print("="*70)
print("📺 유튜브 자막 추출 시작")
print("="*70)

youtube_videos = video_df[video_df['type'] == 'youtube'].copy()
print(f"처리 대상: 유튜브 비디오 {len(youtube_videos)}개\n")

youtube_transcript_results = []

for idx, row in tqdm(youtube_videos.iterrows(), total=len(youtube_videos), desc="유튜브 자막"):
    post_id = row['post_id']
    url = row['url']
    video_id = row.get('youtube_video_id', '')
    
    if not video_id:
        youtube_transcript_results.append({'post_id': post_id, 'url': url, 'video_id': '', 'transcript': '', 'language': '', 'status': 'no_video_id'})
        continue
    
    try:
        try:
            transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=['ko'])
            language = 'ko'
        except:
            try:
                transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])
                language = 'en'
            except:
                transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
                language = 'auto'
        full_transcript = ' '.join([item['text'] for item in transcript_list])
        youtube_transcript_results.append({'post_id': post_id, 'url': url, 'video_id': video_id, 'transcript': full_transcript, 'language': language, 'status': 'success'})
    except TranscriptsDisabled:
        youtube_transcript_results.append({'post_id': post_id, 'url': url, 'video_id': video_id, 'transcript': '', 'language': '', 'status': 'transcripts_disabled'})
    except NoTranscriptFound:
        youtube_transcript_results.append({'post_id': post_id, 'url': url, 'video_id': video_id, 'transcript': '', 'language': '', 'status': 'no_transcript'})
    except Exception as e:
        youtube_transcript_results.append({'post_id': post_id, 'url': url, 'video_id': video_id, 'transcript': '', 'language': '', 'status': f'error: {str(e)[:50]}'})

if youtube_transcript_results:
    youtube_df_result = pd.DataFrame(youtube_transcript_results)
    youtube_df_result.to_csv('/content/pmi_data/results/youtube_transcript_results.csv', index=False, encoding='utf-8-sig')
    success = (youtube_df_result['status'] == 'success').sum()
    no_transcript = (youtube_df_result['status'].isin(['no_transcript', 'transcripts_disabled'])).sum()
    print(f"\n✅ 유튜브 자막 추출 완료!")
    print(f"   - 자막 있음: {success}/{len(youtube_videos)}")
    print(f"   - 자막 없음: {no_transcript}/{len(youtube_videos)}")
    print(f"💾 저장: youtube_transcript_results.csv")

## [셀 8] Whisper 음성인식
**예상 시간: 25-35분** ☕

In [ ]:
import whisper
import subprocess

print("="*70)
print("🎙️ Whisper 음성인식 처리 시작")
print("="*70)

print("🔧 Whisper 모델 로딩 중...")
whisper_model = whisper.load_model("base")
print("✅ Whisper 준비 완료!\n")

youtube_no_transcript = youtube_df_result[youtube_df_result['status'] != 'success']['video_id'].tolist()
naver_video_items = naver_videos[['post_id', 'url']].to_dict('records')

print(f"처리 대상:")
print(f"  - 자막 없는 유튜브: {len(youtube_no_transcript)}개")
print(f"  - 네이버 비디오: {len(naver_video_items)}개")
print(f"  총 {len(youtube_no_transcript) + len(naver_video_items)}개\n")

whisper_results = []

if youtube_no_transcript:
    print("📺 자막 없는 유튜브 비디오 Whisper 처리...")
    for video_id in tqdm(youtube_no_transcript, desc="유튜브 Whisper"):
        try:
            audio_path = f"/content/pmi_data/temp_videos/yt_{video_id}.mp3"
            cmd = ['yt-dlp', '-x', '--audio-format', 'mp3', '-o', audio_path, f'https://www.youtube.com/watch?v={video_id}']
            result = subprocess.run(cmd, capture_output=True, text=True, timeout=120)
            
            if os.path.exists(audio_path):
                result = whisper_model.transcribe(audio_path, language='ko')
                transcript = result['text']
                whisper_results.append({'video_id': video_id, 'type': 'youtube', 'transcript': transcript, 'status': 'success'})
                os.remove(audio_path)
            else:
                whisper_results.append({'video_id': video_id, 'type': 'youtube', 'transcript': '', 'status': 'download_failed'})
        except Exception as e:
            whisper_results.append({'video_id': video_id, 'type': 'youtube', 'transcript': '', 'status': f'error: {str(e)[:50]}'})
        time.sleep(1)

if len(naver_video_items) > 0:
    print("\n📹 네이버 블로그 비디오 Whisper 처리...")
    for item in tqdm(naver_video_items, desc="네이버 Whisper"):
        post_id = item['post_id']
        url = item['url']
        try:
            video_path = f"/content/pmi_data/temp_videos/naver_{post_id}.mp4"
            headers = {'User-Agent': 'Mozilla/5.0', 'Referer': 'https://blog.naver.com/'}
            response = requests.get(url, headers=headers, timeout=60, stream=True)
            
            if response.status_code == 200:
                with open(video_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                result = whisper_model.transcribe(video_path, language='ko')
                transcript = result['text']
                whisper_results.append({'post_id': post_id, 'url': url, 'type': 'naver_blog', 'transcript': transcript, 'status': 'success'})
                os.remove(video_path)
            else:
                whisper_results.append({'post_id': post_id, 'url': url, 'type': 'naver_blog', 'transcript': '', 'status': 'download_failed'})
        except Exception as e:
            whisper_results.append({'post_id': post_id, 'url': url, 'type': 'naver_blog', 'transcript': '', 'status': f'error: {str(e)[:50]}'})
        time.sleep(1)

if whisper_results:
    whisper_df = pd.DataFrame(whisper_results)
    whisper_df.to_csv('/content/pmi_data/results/whisper_transcript_results.csv', index=False, encoding='utf-8-sig')
    success = (whisper_df['status'] == 'success').sum()
    print(f"\n✅ Whisper 음성인식 완료! 성공: {success}/{len(whisper_results)}")
    print(f"💾 저장: whisper_transcript_results.csv")

## [셀 9] 결과 통합 및 최종 저장

In [ ]:
print("="*70)
print("📊 결과 통합 및 최종 저장")
print("="*70)

total_images = len(image_ocr_df)
successful_images = (image_ocr_df['status'] == 'success').sum()
images_with_text = (image_ocr_df['ocr_text'].str.len() > 0).sum()

total_videos = len(video_df)
video_frame_success = len(video_frame_df) if 'video_frame_df' in locals() else 0
youtube_success = (youtube_df_result['status'] == 'success').sum() if 'youtube_df_result' in locals() else 0
whisper_success = (whisper_df['status'] == 'success').sum() if 'whisper_df' in locals() else 0

print(f"""
📊 최종 처리 결과:

1️⃣ 이미지 OCR:
   - 전체: {total_images}개
   - 성공: {successful_images}개 ({successful_images/total_images*100:.1f}%)
   - 텍스트 추출: {images_with_text}개

2️⃣ 비디오 처리:
   - 전체: {total_videos}개
   - 프레임 OCR: {video_frame_success}개
   - 유튜브 자막: {youtube_success}개
   - Whisper 음성인식: {whisper_success}개

📁 생성된 파일:
   1. image_ocr_results.csv
   2. video_frame_ocr_results.csv
   3. youtube_transcript_results.csv
   4. whisper_transcript_results.csv
""")

!cd /content/pmi_data/results && zip -r pmi_ocr_results.zip *.csv
print("✅ 결과 파일 압축 완료: pmi_ocr_results.zip")

## [셀 10] 결과 다운로드

In [ ]:
from google.colab import files

print("📥 결과 파일 다운로드 중...")
files.download('/content/pmi_data/results/pmi_ocr_results.zip')

print("\n🎉 모든 처리 완료!")
print("압축 파일을 다운로드하여 로컬에서 압축을 풀고 확인하세요.")
print("\n📊 다음 단계: 원본 크롤링 데이터와 병합하여 분석 시작!")